<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/energyopt/ThermalEnergyAndNeqSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TESPy and NeqSim interacations
We start by installing the TESTpy and neqsim python packages.

See: https://tespy.readthedocs.io/en/main/first_steps.html


In [2]:
%%capture
!pip install tespy
!pip install neqsim

In [3]:
from tespy.networks import Network

# create a network object with water as fluid
fluid_list = ['water']
my_plant = Network(fluids=fluid_list)

# set the unitsystem for temperatures to °C and for pressure to bar
my_plant.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

In [6]:
from tespy.components import (
    Sink, Source, Pipe, Valve, HeatExchangerSimple)

# sources & sinks (central heating plant)
so = Source('heat source output')
si = Sink('heat source input')

# consumer
cons = HeatExchangerSimple('consumer')
cons.set_attr(Q=-10000, pr=0.98)  # Q in W
val = Valve('valve')
val.set_attr(pr=1)  # pr - pressure ratio (output/input pressure)

# pipes
pipe_feed = Pipe('pipe_feed')
pipe_back = Pipe('pipe_back')

pipe_feed.set_attr(ks=0.0005,  # pipe's roughness in meters
                   L=100,  # length in m
                   D=0.06,  # diameter in m
                   kA=10,  # area independent heat transfer coefficient kA in W/K
                   Tamb=10)  # ambient temperature of the pipe environment (ground temperature)
pipe_back.set_attr(ks=0.0005,
                   L=100,
                   D=0.06,
                   kA=10,
                   Tamb=10)

In [5]:
from tespy.connections import Connection

# connections of the disctrict heating system
so_pif = Connection(so, 'out1', pipe_feed, 'in1')

pif_cons = Connection(pipe_feed, 'out1', cons, 'in1')
cons_val = Connection(cons, 'out1', val, 'in1')

val_pib = Connection(val, 'out1', pipe_back, 'in1')
pib_si = Connection(pipe_back, 'out1', si, 'in1')

# this line is crutial: you have to add all connections to your network
my_plant.add_conns(so_pif, pif_cons, cons_val, val_pib, pib_si)

so_pif.set_attr(T=90, p=15, fluid={'water': 1})
cons_val.set_attr(T=60)

In [7]:
my_plant.solve(mode='design')
my_plant.print_results()

iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 1.44e+06 | 2.03e+00 | 2.60e+06 | 2.03e+05 | 0.00e+00
2	| 1.02e+04 | 1.64e-02 | 8.29e+03 | 8.28e+04 | 0.00e+00
3	| 6.47e+02 | 5.35e-05 | 7.84e-01 | 7.10e+03 | 0.00e+00


4	| 2.21e-01 | 6.71e-08 | 1.78e-03 | 2.47e+00 | 0.00e+00
5	| 8.19e-08 | 3.47e-14 | 9.88e-10 | 9.08e-07 | 0.00e+00
--------+----------+----------+----------+----------+---------
Total iterations: 5, Calculation time: 0.7 s, Iterations per second: 7.22
##### RESULTS (Pipe) #####
+-----------+-----------+----------+----------+----------+----------+----------+----------+----------+
|           |         Q |       pr |     zeta |        D |        L |       ks |       kA |     Tamb |
|-----------+-----------+----------+----------+----------+----------+----------+----------+----------|
| pipe_feed | -7.89e+02 | 1.00e+00 | 4.68e+06 | 6.00e-02 | 1.00e+02 | 5.00e-04 | 1.00e+01 | 1.00e+01 |
| pipe_back | -4.93e+02 | 1.00e+00 | 5.16e+06 | 6.00e-02 | 1.00e+02 | 5.00e-04 | 1.00e+01 | 1.00e+01 |
+-----------+-----------+----------+----------+----------+----------+----------+----------+----------+
##### RESULTS (HeatExchangerSimple) #####
+----------+-----------+----------+----------+-----+-----+----